# Homework 01
## Notebook 1

Group's members:

    Crnigoj Gabriele 134176
    Ferraro Tommaso 132998
    Stinat Kevin 134905
    
<font color='green'>Note</font>: in red we write some comment for each exercise. At the end of the Notebook 02 it can be found our conclusion


# Clustering the top 100 Greatest Movies of All Time (Part 1)

IMBDb is a website containing movies rating. In particular, a set of top 100 greatest movies of all time is available at the URL http://www.imdb.com/list/ls055592025/ . This data is contained in form of a HTML page.\
Our ultimate goal will be to automatically fetch the data from that page and to process it in order to cluster the films.

## Step 1: Getting the data
In this phase, the goal is to scrape the data from the web and making a local copy of the relevant movie synopses for the following phases. In order to do that we will use the `requests` library for interacting with the web service and the `beautifulsoup` library for parsing html web pages and navigating into their content.

### Interacting with a web site
The `requests` library allows to interact with a web site from your Python script. In particular, it allows to download the content of a web page or a web service, given its address (more precisely its URL).\
\
In its essence the `requests` use one `http` method to issue a request to the web server and obtaines a response. This is the basic of the web interaction. In the following example we are interacting with the home page of the University of Udine, getting its content.

In [1]:
import requests

In [2]:
r = requests.get('https://www.uniud.it')
print(r.status_code)

200


The `r` object is the response returned from the web server that is managing the page we have requested. In particular it is relevant to look whether the request was successfully honoured by the web server or some error occured. To do so, the `r.status_code` attribute can be inspected. It is a code, whose value is 200 if everything went fine (actually it is < `400` but we can skip a more detailed explanation now) or it might a value greater than or equal to `400` if something went wrong (all of you probably have experienced the 404 error `“Page not Found”` ).

In [3]:
request = requests.get('http://www.imdb.com/list/ls055592025/') 
print(request.status_code)

200


<font color='red'>**Comment:**</font> The value of the response is 200, so the interaction with the website went fine.

The actual content of the page is typically encoded in the `HTML` language. This can be inspected by looking at the `r.headers` field of the response, i.e.,\
\
`r.headers['content-type']`\
\
which returns `text/html` when dealing with a regular website. In this case the content is of this type, the actual page content is available as the `r.text` or the `r.content` fields of the response. In general, however the interaction through the `http` protocol is possible also with web services, which usually returns data in different formats. For example, the site `http://www.omdbapi.com` allows to get the movie information from the IMDb site (Internet Movie Database) site as a web service, however it will need to be registred to that website.

In [4]:
print(request.headers['content-type'])

text/html;charset=UTF-8


<font color='red'>**Comment:**</font> in this case the content is of this type, the actual page content is available as the r.text
or the r.content fields of the response.\
Once the content has been retrieved, if it is in form of an HTML page it could be manipulated through the beautifulsoup library. Through the page object, now it is possible to navigate and search the HTML content, retrieving a specific fragment of the page itself (more later).

Once the content has been retrieved, if it is in form of an HTML page it could be manipulated through the `beautifulsoup` library. In particular the content can be parsed in this way:\
\
`
import BeautifulSoup from bs4
 page = BeautifulSoup(res.content)
`
\
\
Through the `page` object, now it is possible to navigate and search the HTML content, retrieving a specific fragment of the page itself (more later).

In [5]:
from bs4 import BeautifulSoup
page = BeautifulSoup(request.content)

#page

<font color='red'>**Comment:**</font>\
r.content : show the page in HTML\
BeautifulSoup : Trasform that page into a webpage readible in HTML\
Print(page) : show me the entire content of the page. We don't print it because of the lenght of the print.

### EXERCISE 1

The list of the 100 greatest movie of all the time (you will find it in the file `greatest_movies.xlsx`. Given the data extract the synopsis of each movie and save it to a file whose name is the title of the movie. To get the synopsis, you may use the following function "get_synopsis(id)".
```
def get_synopsis(id):
    res = requests.get('https://www.imdb.com/title/{movie_id}/plot
    summary'.format(movie_id=id))
    if res.status_code != 200:
        return None
    page = BeautifulSoup(res.content)
    synopsis = page.select("#plot-synopsis-content")
    return synopsis[0].text.strip()
```
This function will interact with the `imdb` website getting the description page
of each movie and extracting just the synopsis.\
\
**Note**: it would be better to save the movies files inside a directory instead of creating them in the current directory. For this purpose, you can refer to the `os` library that allows to interact with the operating system. In particular `os.mkdir(name)` will create a new directory whose name is name . In order to create the directory only if it does not exist previously, you can use the `os.path.exists(name)` method that checks if the directory (or a single file already exists).

In [6]:
import pandas as pd
import os
from tqdm import tqdm 
import time

<font color='red'>**Comment:**</font> First of all, we read the data from the file '.xlsx' and create a DataFrame\
Then we clean and improve the readability of the DataFrame, deleting the 'Unnamed: 0' column that became from an error due to the reading of the '.xlsx' file\
At the end we show the resul of that operation, displaying the first five rows of the DataFrame

In [7]:
file_movies = 'greatest_movies.xlsx'
greatest_movies = pd.read_excel(file_movies)

greatest_movies.index.name = 'index'
greatest_movies = greatest_movies.drop(columns = ['Unnamed: 0'])

greatest_movies.head()

,title,id
index,,
0,The Godfather,tt0068646
1,The Shawshank Redemption,tt0111161
2,Schindler's List,tt0108052
3,Raging Bull,tt0081398
4,Casablanca,tt0034583


<font color='red'>**Comment:**</font> We define "get_synopsis(id)" that return the synopsis in text form, and after that, we make an example of what this function return

In [8]:
def get_synopsis(id): 
    res = requests.get('https://www.imdb.com/title/{movie_id}/plotsummary'.format(movie_id = id))
    if res.status_code != 200: #check che non mi dia errore
        return None
    page = BeautifulSoup(res.content)
    synopsis = page.select("#plot-synopsis-content")
    return synopsis[0].text.strip()

In [9]:
print(greatest_movies['title'][1])
get_synopsis(greatest_movies['id'][1])

The Shawshank Redemption


'In 1947, Andy Dufresne (Tim Robbins), a banker in Maine, is convicted of murdering his wife and her lover, a golf pro. Since the state of Maine has no death penalty, he is given two consecutive life sentences and sent to the notoriously harsh Shawshank Prison. Andy keeps claiming his innocence, but his cold and calculating demeanor leads everyone to believe he did it.Meanwhile, Ellis Boyd Redding (Morgan Freeman), known as Red is being interviewed for parole after having spent 20 years at Shawshank for murder. Despite his best efforts and behavior, Red\'s parole is rejected which doesn\'t phase him all that much. Red is then introduced as the local smuggler who can get inmates anything they want within reason. An alarm goes off alerting all prisoners of new arrivals. Red and his friends bet on whichever new fish will have a nervous break down during his first night in prison. Red places a huge bet on Andy.During the first night, an overweight newly arrived inmate, nicknamed \'\'fat as

<font color='red'>**Comment:**</font> We proceeded by creating the folder in which we want to save the text files and we check that the folder exists, and then we created each file with the title equal to the id of the film (not the title because were born problems with the film's titles that contained punctuation symbols) and at the end we save inside them each synopsis.\
We also use 'tqdm' function to analyze the progress status of the processing of the function\
\
**Note**: if there weren't any kind of error in the films' title, we would have used the function below, paying attention to delete the punctation:

```
for c,movie_id in tqdm(enumerate(greatest_movies['id'])):\
    sinossi=get_synopsis(movie_id)\
    with open(os.path.join(save_path,'{title}.txt'.format(title=greatest_movies['title'][c])),'w') as r:\
        r.write(sinossi)
```

In [10]:
os.mkdir('Best 100 film\'s synopsis')
save_path = 'Best 100 film\'s synopsis/'

In [11]:
os.path.exists('Best 100 film\'s synopsis/')

True

In [12]:
if os.path.exists('Best 100 film\'s synopsis/') == True:
    for movie_id in tqdm(greatest_movies['id']): #movie_id = id of each film
    
        sinossi = get_synopsis(movie_id)
        
        with open(os.path.join(save_path,'{title}.txt'.format(title = movie_id)),'w') as r:
            r.write(sinossi)
else:
    print('Directory not found.')

100%|██████████| 100/100 [01:39<00:00,  1.04it/s]


## Step 2: Data preparation

Textual data needs a proper preparation for later analysis. In particular, the main activities involve the removal of the terms that are too frequent and do not convey significant meaning (such as articles, pronouns, or prepositions). This task is called **stopword removal**.\
\
Afterwards, a sort of normalization of the terms is needed. Indeed, terms appear in different inflected/derived forms, such as, for example, "am", "are", "is" all refer to the "to be" verb, "car", "cars", "car's", "cars'" all refer to "car".\
**Stemming** is the process of transforming the text so that the root of each stemmed version is used.\
\
Finally, once the text has been properly normalized it should undergo a **tokenization** process to transform the text flow in a list of terms, to be used either directly to represent the document or to be the source of other processing (e.g., -gram building).\
\
The goal of this phase is to get a proper representation of the documents (i.e., the movies) that can be used for the following clustering phase.\
\
To help in this phase we will make use of the `nltk` library (Natural Language Toolkit, https://www.nltk.org), a very comprehensive library for performing textual analysis.

### EXERCISE 2

The  `nltk ` library provides a bunch of corpora, which are relevant textual sources for different purposes (e.g., 25,000 free books of the Gutenberg project, >10,000 documents of the Reuters news, ...).\
\
One particular corpus is the  `stopwords ` one, which contains the stopwords (in different languages), which is available in the  `nltk.corpus ` package. \
\
Load the whole  `stopwords ` corpus of the English language in a  `stopwords ` variable. For suggestion you can refer to https://www.nltk.org/book/ch02.html. \
\
**Note**: in order to use the corpora, you are required to download them, therefore you should issue the nltk.download(corpus_name) command for this purpose.

In [ ]:
import nltk

In [ ]:
nltk.download('stopwords')

In [ ]:
nltk.download('punkt')

### EXERCISE 3

The  `nltk.sent_tokenize(text) ` and the  `nltk.word_tokenize(sentence) ` are two notable functions for tokenizing a text to sentences and a sentence to word, respectively. It would be advisable also to normalize the tokens by transforming them to lowercase.\
\
A stemmer, instead, is a function that will transform each word to its root. A very popular one is the so-called *Snowball stemmer*, which is available from the package `nltk.stem.snowball` as `SnowballStemmer`. The `SnowballStemmer` constructor takes a string as its argument, which specifies the stemming language ( `"english"`in our case). 

Write a function `tokenize_and_stem(text)` that, given the text of the synopsis tokenizes its content, stems it, removes the stopwords and everything that is not a word (i.e., everything whose name does not start with a letter, can you write a regular expression for that?). 

**Note**: maybe you should download the `'punkt'` component for tokenizing sentences as `nltk.download('punkt')`.

In [ ]:
from nltk.stem.snowball import SnowballStemmer

In [ ]:
stopwords = nltk.corpus.stopwords.words('english') 
stemmer = SnowballStemmer('english')

In [ ]:
def tokenize_and_stem(synopsis):
    
    lists_tokenized = []
    
    synopsis = synopsis.lower() #Don't want capital letters
    
    sent_tokens = nltk.sent_tokenize(synopsis) #tokenizes the synopsis into sentences in a list called sent_tokenized
    
    for sentence in sent_tokens:
        list_tokenized = nltk.word_tokenize(sentence) #tokenizes the words in a list called list_tokenized
        lists_tokenized.extend(list_tokenized)
        
    list_stemmed = list(stemmer.stem(i) for i in lists_tokenized) #list with every token stemmed (the root of the word)
    
    only_words = list(filter(lambda x: x.isalpha(), list_stemmed)) #it mantainces only words and nothing else
    
    without_stopwords = list(filter(lambda x: x not in stopwords, only_words)) 
    #if lambda is true, keep the word in the liste, otherwise remove it
    
    return without_stopwords

<font color='red'>**Comment:**</font> after the importation of the above libraries, we wrote the function above which tokenize and do the stemming of a text . Now we can apply it to every file that we have saved before into a '.txt file'

### EXERCISE 4

Load all the synopses from the directory into a list of strings.

<font color='red'>**Comment:**</font> After we see that during the exercise 5 we need the names (or the ids, but for a better readable form, we prefer the name of each film) we keep track of names and ids while the function is reading in order the '.txt' file. Then we can assign to the columns of the Dataframe the names of the correspounding film.
For this porpouse we wrote the function "convert" and thet we print the result of the application of that function.

In [ ]:
def convert(dataframe, movie_id):
    
    return greatest_movies.loc[greatest_movies['id'] == movie_id]['title'].values[0]

In [ ]:
string_list = []
list_movie_id = []
list_movie_title = []

for movie_id in tqdm(greatest_movies['id']): #movie_id = id of each film
    
    with open(os.path.join('Best 100 film\'s synopsis/','{title}.txt'.format(title = movie_id)),'r') as file:  
        clean_token_list = tokenize_and_stem(file.read())
        string_list.append(' '.join(clean_token_list))
        
        list_movie_id.append(movie_id)
        list_movie_title.append(convert(greatest_movies, movie_id))

In [ ]:
string_list

In [ ]:
print(list_movie_id)

In [ ]:
print(list_movie_title)

<font color='red'>**Comment:**</font> In the next step, we read the '.txt' files which contain the synopsis of each film. Then we save every synosy without the stopwords into a list of strings, to be ready for the next step: the start of the clustering. In the we show the result of the previous function.\

### EXERCISE 5

Now we have to define any document (i.e., a movie synopsis) as a vector of features. In particular, we want to use the *tf.idf* to characterize each term (Term-Frequency Inverse document frequency), so that we will obtain a *tf.idf matrix*.

In order to do that, we can make use of the `sklearn` library (Sci-Kit learn), a general purpose machine learning library. In particular, the tf.idf is available in the package `sklearn.feature_extraction.text` as the class `TfidfVectorizer`.

In general, the `sklearn` classes have two peculiar methods `fit()` and `transform()` which let the data be learned and the model applied, respectively. In our case, the two operations can be combined in one, using the `fit_transform()` method.

When a `TfidfVectorizer` is created a couple of parameters have to be provided:
- `max_df` : the maximum frequency within the documents a given term can have to be used in the tf-idf matrix; a reasonable     value is (or 0.8 ). If the term appears everywhere it probably carries little meaning.
- `min_df` : the minimum value for document frequencies; using values that appear too seldomly also carries little meaning, a reasonable value is (or 0.2).
- `max_features` : the maximum terms (i.e., features) to be considered.
- `ngram_range` : whether to look at single words or sequences of more than one token.
- `tokenizer` : a function that will perform tokenization.

Once the vectorizer is in place, the terms employed are possibly available through the `get_feature_names(`) method.

The matrix returned from the vectorizer is a *document x terms* whose values are the *tf_df* metric computed for each pair.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

<font color='red'>**Comment:**</font> First we import the TfidfVectorizer class.
- The **sklearn.feature_extraction** module deals with feature extraction from raw data.
- The **sklearn.feature_extraction.text** submodule gathers utilities to build feature vectors from text documents.
- The class **sklearn.feature_extraction.text.TfidfVectorizer** convert a collection of raw documents to a matrix of TF-IDF features.
- The input is expected to be the **sequence strings or bytes items** that are expected to be analyzed directly.

Then we create a object which is associated with the class TfidfVectorizer.
- min_df : the minimum value for document frequencies
- max_features : the maximum terms (i.e., features) to be considered

In [ ]:
tfidf_vectorizer_1 = TfidfVectorizer(max_df = 0.8, min_df = 0.2) 

<font color='red'>**Comment:**</font> Knowing about the short theory which we report below:
- **fit_trasform()**: is a method of the TfidfVectorizer class. It is a quicker way to execute subsequently the functions fit() and then trasform().
- **fit()**: fit the vectorizer to the training data and save the vectorizer to a variable.
- **trasform()**: use the variable output from fit() to transformer validation/test data into a matrix (Returns: scipy.sparse.csr.csr_matrix).

We call **vectorization** the general process of turning a collection of text documents into numerical feature vectors.
Documents are described by word occurrences while completely ignoring the relative position information of the words in the document.

**Note:** We not call for the next feature because we use their default value:
- tokenizer : callable or None (default=None)
- ngram_range : tuple (min_n, max_n) (default=(1, 1))
- max_features : int or None (default=None)

Now we can procede to the next steps, often controlling the correct result of each operation (for example we chack the shape of the matrix and the DataFrame)

In [ ]:
#tfidf_vectorizer_vectors is the link to the matrix before we create the DataFrame 

tfidf_vectorizer_vectors_1 = tfidf_vectorizer_1.fit_transform(string_list)
tfidf_vectorizer_vectors_1

In [ ]:
#it saves the correct matrix with 100 columns (documents) and 475 rows (words). Notice that the vectors are vertical.
#We can also see the dimension of the sparse matrix from the cell above

tfidf_vectorizer_vectors_1.shape

In [ ]:
tf_idfmatrix_1 = pd.DataFrame(tfidf_vectorizer_vectors_1.T.todense(), index = tfidf_vectorizer_1.get_feature_names())

<font color='red'>**Comment:**</font> We want to transform the matrix into a Pandas DataFrame with horizontal vectors instead of the vertical ones in the tfidf_vectorizer_vectors. It can be considered as sparse matrix as each vector contains few terms that will have a non-zero value and it is their **idf** value.

- **.todense()** return a dense representation of this matrix. **tfidf_vectorizer_vectors is a sparse matrix**. It means that  almost all entries are zeros and it wastes a lot of memory. Ideed, If i call tfidf_vectorizer_vectors i'll obtain a link to the matrix which is saved in memory.
- **.get_feature_names().** In the original matrix every column and row is represented by a number. The function is an array mapping from feature integer (columns and rows) indices to feature name.

In [ ]:
tf_idfmatrix_1.shape

In [ ]:
tf_idfmatrix_1.head()

<font color='red'>**Comment:**</font> As we have said before, we associate each column at its movie' id or name. We propose an example of both association, but we use the second one bucause of the better readibility.

In [ ]:
tf_idfmatrix_1.columns = list_movie_id
tf_idfmatrix_1.head()

In [ ]:
tf_idfmatrix_1.columns = list_movie_title
tf_idfmatrix_1.head()

<font color='red'>**Comment:**</font> To check our results we used an other method in which we use the **tokenize_and_stem** function into the **TfidfVectorizer**, passing a list of string in which there are all the synopsis readed into the '.txt' files. As we can see below we had used the **.toarray()** function to obtain a condence matrix. But for a more readability of the data, we think that it's better to use a dataframe.

In [ ]:
tfidf_vectorizer_2 = TfidfVectorizer(max_df=0.8, min_df=0.2, analyzer='word', tokenizer = tokenize_and_stem)

In [ ]:
list_synopsis = []

for movie_id in tqdm(greatest_movies['id']): #movie_id = id of each film
    
    with open(os.path.join('Best 100 film\'s synopsis/','{title}.txt'.format(title = movie_id)),'r') as file:  
        list_synopsis.append(file.read())      

In [ ]:
tfidf_vectorizer_vectors_2 = tfidf_vectorizer_2.fit_transform(list_synopsis)
tfidf_vectorizer_vectors_2

<font color='red'>**Comment:**</font> The parameter **max_feature** select the first n-elements order by decrescent frequency. But we think that we had already too few words to manage our data so we preferred not to use this parameter. However we made an example of the **max_feature** usage.

Notice that we arrive at the same result (same number of rows (or words))

In [ ]:
tfidf_vectorizer_3 = TfidfVectorizer(max_features = 350, max_df=0.8, min_df=0.2, analyzer='word', tokenizer = tokenize_and_stem)

tfidf_vectorizer_vectors_3 = tfidf_vectorizer_3.fit_transform(list_synopsis)
tfidf_vectorizer_vectors_3

In [ ]:
tfidf_vectorizer_vectors_3.toarray()

### EXERCISE 6

We want to save the results of this preparation phase, which is embedded in the `tf_idfmatrix` object. To do that, we can use the `joblib` library (available as `sklearn.externals.joblib` , which allows to save a Python object to a (binary) file.
\
\
The function `joblib.dump(object, filename)` writes the content to a file, whereas `pickle.load(filename)` reads from a file and returns the loaded object.            

In [ ]:
import joblib
import pickle

In [ ]:
from tempfile import mkdtemp
import os
os.mkdir('tf_idfmatrix')
savedir = 'tf_idfmatrix'

In [ ]:
filename = os.path.join(savedir, 'tf_idfmatrix_file')

with open(filename,'wb') as r:
    joblib.dump(tf_idfmatrix_1, r)

In [ ]:
with open(filename,'rb') as r:
    example = pickle.load(r)
    
example

In [ ]:
type(example)

<font color='red'>**Comment:**</font> In order to save the matrix we have to use Joblib library to save it into a directory called 'tf_idmatrix' and we create a file called in the same way to save our matrix.

We use "wb" because it is a binary file. If we open the file, we can read it bacause it has a different way of reading than a normal file. For this reason, we load it with joblib.

We open the sam file with pickle, to make an example of how to use that function. 

Then we can open a new Notebook and test if the dump of the binary file went fine. 

<font color='red'>**Comment:**</font> we notice that using pickle to load the data we receive a nunpy.ndarry, so this function not suit our work because we want to read the entire matrix that we dumped into the previous file.